# 1.Dataloader与Dataset

Q:如何用torch.utils.data.DataLoader构建可迭代的数据装载器？
- 这是个类：`torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None, generator=None)`
- dataset: Dataset类，决定数据从哪读取及如何读取
- batchsize：批大小
- num_works：是否多进程读取数据
- shuffle：每个epoch是否乱序
- drop_last：当样本数不能被batchsize整除时，是否舍弃最后一批数据

Q:torch.utils.data.Dataset的功能是什么？
- Dataset抽象类，所有自定义的Dataset需要继承它，并且复写__getitem__()
- getitem:接收一个索引，返回一个样本
- 要如何读取样本，是用户需要自定义的
- ![](http://anki190912.xuexihaike.com/20200920111241.png?imageView2/2/h/150)

Q:DataLoader数据读取三问：读哪些，从哪读，怎么读
- 1.读哪些？Sampler输出的Index
- 2.从哪读？Dataset中的data_dir
- 3.怎么读？Dataset的getitem

Q:DataLoader数据读取流程是怎样？
- ![](http://anki190912.xuexihaike.com/20200920115859.png)
- 1.在for循环中使用DataLoader
- 2.根据是否采用多进程还是单进程的DataLoaderIter
- 3.使用Sampler获得index索引
- 4.拿到index索引然后给DatasetFetcher
- 5.在这里调用Dataset
- 6.Dataset根据给定的索引在getitem中在硬盘里读取实际的图像和和标签
- 7.读取了一个batchsize大小的图像后，通过collate_fn进行整理成一个Batch Data形式，然后输入到模型中训练

# 2.transforms与normalize

Q:torchvision（计算机视觉工具包）中有哪些模块以及相应的作用？
- torchvision.transforms：常用的图像预处理方法
- torchvision.datasets：常用数据集的dataset实现，MNIST,CIFAR-10,ImageNet等
- torchvision.model：常用的模型预训练，AlexNet,VGG,ResNet,GoogLeNet等

Q:pytorch的transform机制是在哪一步做？
- ![](http://anki190912.xuexihaike.com/20200920150244.png)

Q:如何逐channel地对图像进行标准化？
- `transforms.Normalize(mean, std, inplace=False)`
- output = (input - mean) / std
- mean：各通道的均值
- std：各通道的标准差
- inplace：是否原地操作

# 3.transforms图像增强（一）
Q:如何从图像中心裁剪图片？
- `transforms.CenterCrop`
- size：所需裁剪图片尺寸

Q:如何从图片中随机裁剪出尺寸为size的图片？
- `transforms.RandomCrop(size, padding=None, pad_if_needed=False, fill=0, padding_mode='constant')`
- size：所需裁剪图片尺寸
- padding：设置填充大小
    - 当为a时，上下左右均填充a个像素
    - 当为(a,b)时，上下填充b个像素，左右填充a个像素
    - 党委(a,b,c,d)时，左，上，右，下分别填充a,b,c,d
- pad_if_need：若图像小于设定size，则填充
- padding_mode：填充模式，有4种模式
    - constand：像素值由fill设定
    - edge：像素值由图像边缘像素决定
    - reflect：镜像填充，最后一个像素不镜像,e.g.：[1,2,3,4]->[3,2,1,2,3,4,3,2]
    - symmetric：镜像填充，最后一个像素镜像，e.g.：[1,2,3,4]->[2,1,1,2,3,4,4,3]
- fill：constant时，设置填充的像素值，(R,G,B) or (Gray)

Q:如何随机大小、长宽比裁剪图片？
- `RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(3/4, 4/3), interpolation)`
- size：所需裁剪图片尺寸
- scale：随机裁剪面积比例，默认(0.08, 1)
- ratio：随机长宽比，默认(3/4, 4/3)
- interpolation：插值方法
    - PIL.Image.NEAREST
    - PIL.Image.BILINEAR
    - PIL.Image.BICUBIC

Q:FiveCrop和TenCrop的功能是什么？
- `transforms.FiveCrop(size)`
- `transforms.TenCrop(size, vertival_flip=False)`
- 功能：在图像的上下左右以及中心裁剪出尺寸为size的5张图片，TenCrop对这5张图片进行水平或者垂直镜像获得10张图片
- size：所需裁剪图片尺寸
- vertical_flip：是否垂直翻转

Q:如何按照概率水平（左右）或垂直（上下）翻转图片？
- `RandomHorizontalFlip(p=0.5)`
- `RandomVerticalFlip(p=0.5)`
- p：翻转概率

Q:如何随机旋转图片？
- `RandomRotation(degress, resample=False, expand=False, center=None)`
- degree：旋转角度
    - 当为a时，在(-a, a)之间选择旋转角度
    - 当为(a, b)时，在(a,b)之间选择旋转角度
- resample：重采样方法
- expand：是否扩大图片，以保持原图信息
- center：旋转点设置，默认中心选择

# 4.transforms图像增强(二)
Q:如何对图像边缘进行填充？
- `transforms.Pad(padding, fill=0, padding_mode='constant')`
- padding：设置填充大小
    - 当为a时，上下左右均填充a个像素
    - 当为(a,b)时，上下填充b个像素，左右填充a个像素
    - 党委(a,b,c,d)时，左，上，右，下分别填充a,b,c,d
- padding_mode：填充模式，有4种模式
    - constand：像素值由fill设定
    - edge：像素值由图像边缘像素决定
    - reflect：镜像填充，最后一个像素不镜像,e.g.：[1,2,3,4]->[3,2,1,2,3,4,3,2]
    - symmetric：镜像填充，最后一个像素镜像，e.g.：[1,2,3,4]->[2,1,1,2,3,4,4,3]
- fill：constant时，设置填充的像素值，(R,G,B) or (Gray)

Q:如何调整亮度、对比度、 饱和度和色相？
- `transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0)`
- brightness：亮度调整因子
    - 当为a时，从[max(0, 1-a), 1+a]中随机选择
    - 当为(a,b)时，从[a,b]中
- contrast：对比度参数，同brightness
- saturation：饱和度参数，同brightness
- hue：色相参数
    - 当为a时，从[-a, a]中选择参数，注：0<=a<=0.5
    - 当为(a,b)时，从[a,b]中选择参数，注：-0.5 <= a <= b <= 0.5

Q:如何对把彩色图像转换为灰度图像？
- `Grayscale(num_output_channels)`
- `RandomGrayscale(num_output_channels, p=0.1)`
- num_output_channels：输出通道数，只能设1或3
- p：概率值，图像被转换为灰度图的概率

Q:如何对图像进行仿射变换？
- `RandomAffile(degrees, translate=None, scale=None, shear=None, resample=False, fillcolor=0)`
- 仿射变换是二维的线性变换，由五种基本原子变换构成，分别是选择、平移、缩放、错切和翻转
- degrees：旋转角度设置
- translate：平移区间设置，如(a,b)，a设置宽(width)，b设置高(height)，图像在宽维度平移的区间为-img_width * a < dx < img_width * a
- scale：缩放比例（以面积为单位）
- fill_color：填充颜色设置
- shear：错切角度设置，有水平错切和垂直错切
    - 若为a，则仅在x轴错切，错切角度为(-a, a)之间
    - 若为(a, b)，则a设置x轴角度，b设置y的角度
    - 若为(a, b, c, d)，则a, b设置x轴角度，c, d设置y轴角度
- resample：重采样方式，有NEAREST、BILINEAR、BICUBIC

Q:如何对图像进行随机遮挡？
- `RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=Flase)`
- p：概率值，执行该操作的概率
- scale：遮挡区域的面积
- ratio：遮挡区域长宽比
- value：设置遮挡区域的像素值，(R,G,B) or (Gray) or 任意字符串
- 参考文献：《Random Erasing Data Augmentation》
- 注意：这个接收的是Tensor，所以在上一步需要执行`transforms.ToTensor()`

Q:如何自定义lambda方法？
- `transforms.Lambda(lambd)`
- lambd：lambda匿名函数
- lambda [arg1 [, arg2, ..., argn]]:expression
- ```python
transforms.TenCrop(200, vertical_flip=True)
transforms.Lambda(lambda crops: torch.stack([transforms.Totensor()(crop) for crop in crops]))
```

Q:如何从一系列transforms方法中随机挑选一个？
- `transforms.RandomChoice([transforms1, transforms2, transforms3])`

Q:如何依概率执行一组transforms操作？
- `transforms.RandomApply([transforms1, transforms2, transforms3], p=0.5)`

Q:如何对一组transforms操作打乱顺序？
- `transforms.RandomOrder([transforms1, transforms2, transforms3])`

Q:用户如何自定义transforms方法？
- 调用方法：
- ```python
class Compose(object):
    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img
```
- 要素：
    - 1.仅接收一个参数，返回一个参数
    - 2.注意上下游的输出与输入
- 通过类实现多参数传入：
- ```python
class YourTransforms(object):
    def __init__(self, ...):
        ...
    def __call__(self, img):
        ...
        return img
```